# TensorFlow!

In [1]:
import tensorflow
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import tensorflow.keras.utils as utils
import tensorflow.keras.optimizers as optimizers

client1_model = tensorflow.keras.models.load_model(
    'model_weight/global_model.h5')
client2_model = tensorflow.keras.models.load_model(
    'model_weight/global_model.h5')
client3_model = tensorflow.keras.models.load_model(
    'model_weight/global_model.h5')

# It's training time!

In [2]:
import numpy
import tensorflow.keras.callbacks as callbacks


def get_dataset():
	container = numpy.load('dataset.npz') # Download dataset.npz from the link in Readme.md
	b, v = container['b'], container['v']
	v = numpy.asarray(v / abs(v).max() / 2 + 0.5, dtype=numpy.float32) # normalization (0 - 1)
	return b, v

x_train, y_train = get_dataset()

In [3]:
# Split dataset for two clients
# pip install -U scikit-learn
from sklearn.model_selection import train_test_split

client1_x, client2_x, client1_y, client2y = train_test_split(x_train, y_train, test_size=0.33, random_state=100)
len(client1_x)
len(client2_x)

495000

In [4]:
# Dellocate variable to save RAM space
del x_train
del y_train

In [5]:
client1_x, client3_x, client1_y, client3y = train_test_split(
    client1_x, client1_y, test_size=0.5, random_state=100)
print(len(client1_x), len(client2_x), len(client3_x))


502500 495000 502500


In [6]:
"""
Training client 1 model
"""
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_filepath = '/checkpoint/f2-1/'
model_checkpointing_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
)
client1_model.fit(client1_x, client1_y,
          batch_size=2048,
          epochs=100,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4), model_checkpointing_callback])


Epoch 1/100
221/221 [==============================] - ETA: 0s - loss: 2.8138e-04

INFO:tensorflow:Assets written to: /checkpoint/f2-1\assets


INFO:tensorflow:Assets written to: /checkpoint/f2-1\assets


221/221 [==============================] - 77s 344ms/step - loss: 2.8138e-04 - val_loss: 2.7824e-04 - lr: 5.0000e-06
Epoch 2/100
221/221 [==============================] - 73s 331ms/step - loss: 2.8082e-04 - val_loss: 2.7905e-04 - lr: 5.0000e-06
Epoch 3/100
221/221 [==============================] - 71s 323ms/step - loss: 2.8029e-04 - val_loss: 2.7859e-04 - lr: 5.0000e-06
Epoch 4/100
221/221 [==============================] - 71s 322ms/step - loss: 2.7998e-04 - val_loss: 2.7974e-04 - lr: 5.0000e-06
Epoch 5/100
221/221 [==============================] - 71s 322ms/step - loss: 2.7958e-04 - val_loss: 2.7949e-04 - lr: 5.0000e-06
Epoch 6/100
221/221 [==============================] - 72s 324ms/step - loss: 2.7927e-04 - val_loss: 2.7887e-04 - lr: 5.0000e-06
Epoch 7/100
221/221 [==============================] - 71s 322ms/step - loss: 2.7889e-04 - val_loss: 2.7949e-04 - lr: 5.0000e-06
Epoch 8/100
221/221 [==============================] - 71s 322ms/step - loss: 2.7851e-04 - val_loss: 2.7913e-

In [7]:
"""
Training client 2 model
"""
checkpoint_filepath = '/checkpoint/f2-2/'
model_checkpointing_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
)
client2_model.fit(client2_x, client2y,
          batch_size=2048,
          epochs=100,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4), model_checkpointing_callback])


Epoch 1/100
218/218 [==============================] - ETA: 0s - loss: 2.8701e-04

INFO:tensorflow:Assets written to: /checkpoint/f2-2\assets


INFO:tensorflow:Assets written to: /checkpoint/f2-2\assets


218/218 [==============================] - 71s 325ms/step - loss: 2.8701e-04 - val_loss: 2.7857e-04 - lr: 5.0000e-06
Epoch 2/100
218/218 [==============================] - 71s 323ms/step - loss: 2.8639e-04 - val_loss: 2.7874e-04 - lr: 5.0000e-06
Epoch 3/100
218/218 [==============================] - 71s 327ms/step - loss: 2.8593e-04 - val_loss: 2.7894e-04 - lr: 5.0000e-06
Epoch 4/100
218/218 [==============================] - 71s 325ms/step - loss: 2.8546e-04 - val_loss: 2.7911e-04 - lr: 5.0000e-06
Epoch 5/100
218/218 [==============================] - 71s 325ms/step - loss: 2.8509e-04 - val_loss: 2.7955e-04 - lr: 5.0000e-06
Epoch 6/100
218/218 [==============================] - 71s 325ms/step - loss: 2.8476e-04 - val_loss: 2.7951e-04 - lr: 5.0000e-06
Epoch 7/100
218/218 [==============================] - 71s 327ms/step - loss: 2.8435e-04 - val_loss: 2.7982e-04 - lr: 5.0000e-06
Epoch 8/100
218/218 [==============================] - 71s 324ms/step - loss: 2.8402e-04 - val_loss: 2.8018e-

In [8]:
"""
Training client 3 model
"""
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_filepath = '/checkpoint/f2-3/'
model_checkpointing_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
)
client3_model.fit(client3_x, client3y,
          batch_size=2048,
          epochs=100,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4), model_checkpointing_callback])


Epoch 1/100
221/221 [==============================] - ETA: 0s - loss: 2.8602e-04

INFO:tensorflow:Assets written to: /checkpoint/f2-3\assets


INFO:tensorflow:Assets written to: /checkpoint/f2-3\assets


221/221 [==============================] - 78s 351ms/step - loss: 2.8602e-04 - val_loss: 2.8209e-04 - lr: 5.0000e-06
Epoch 2/100
221/221 [==============================] - 72s 325ms/step - loss: 2.8543e-04 - val_loss: 2.8276e-04 - lr: 5.0000e-06
Epoch 3/100
221/221 [==============================] - 71s 323ms/step - loss: 2.8502e-04 - val_loss: 2.8241e-04 - lr: 5.0000e-06
Epoch 4/100
221/221 [==============================] - 71s 322ms/step - loss: 2.8459e-04 - val_loss: 2.8293e-04 - lr: 5.0000e-06
Epoch 5/100
221/221 [==============================] - 71s 323ms/step - loss: 2.8417e-04 - val_loss: 2.8314e-04 - lr: 5.0000e-06
Epoch 6/100
221/221 [==============================] - 71s 323ms/step - loss: 2.8381e-04 - val_loss: 2.8343e-04 - lr: 5.0000e-06
Epoch 7/100
221/221 [==============================] - 73s 329ms/step - loss: 2.8351e-04 - val_loss: 2.8326e-04 - lr: 5.0000e-06
Epoch 8/100
221/221 [==============================] - 72s 326ms/step - loss: 2.8314e-04 - val_loss: 2.8350e-

In [9]:
"""
Aggregrate models
"""

client1_weights = client1_model.get_weights()
client2_weights = client2_model.get_weights()
client3_weights = client3_model.get_weights()

client1_weights = numpy.array(client1_weights, dtype=object)
client2_weights = numpy.array(client2_weights, dtype=object)
client3_weights = numpy.array(client3_weights, dtype=object)

client1_weights = 1/3.35 * client1_weights
client2_weights = 1/3.3 * client2_weights
client3_weights = 1/3.35 * client3_weights

aggregated_weights = numpy.add(client1_weights,client2_weights)
aggregated_weights = numpy.add(aggregated_weights, client3_weights)

client1_model.set_weights(aggregated_weights)
client1_model.save_weights("model_weight/federated2_weights", save_format="h5")


AttributeError: 'numpy.ndarray' object has no attribute 'set_weights'